In [29]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import scipy as sp
from scipy.sparse import csr_matrix

In [51]:
def basis_set_generator(tot_sites, N_particles): # Checked OK
    basis = []

    for comb in combinations(range(tot_sites), N_particles):
        state = [0] * tot_sites
        for idx in comb:
            state[idx] = 1
        basis.append(state)
    
    return np.array(basis)


def state_idx_mapping(basis_set): # Checked OK
    return {tuple(state) : i for i, state in enumerate(basis_set)}


def creation_operator(state, i): # Checked OK
    if state[i] == 1:
        return 0, None
    
    res_state = state.copy()
    res_state[i] = 1

    sign = 1
    for j in range(i):
        if res_state[j] == 1:
            sign *= -1
    
    return sign, res_state


def annihilation_operator(state, i): # Checked OK
    if state[i] == 0:
        return 0, None
    
    res_state = state.copy()
    res_state[i] = 0

    sign = 1
    for j in range(i):
        if res_state[j] == 1:
            sign *= -1
    
    return sign, res_state


def hopping_operator(state, initial_position, target_position): # Checked OK
    sign_1, state_1 = annihilation_operator(state, initial_position)

    # if sign_1 == 0:
    #     return 0, None

    sign_2, state_2 = creation_operator(state_1, target_position)

    return sign_1 * sign_2, state_2


def hamiltonian_matrix_generator(basis_set, tot_sites, v, w): # Checked OK
    dim = len(basis_set)
    hamiltonian = np.zeros((dim, dim), dtype=np.float64)

    state_idx_dict = state_idx_mapping(basis_set)

    for i, state in enumerate(basis_set):
        for j in range(tot_sites):
            if state[j] == 0:
                continue

            if j + 1 < tot_sites and state[j + 1] == 0:
                initial_position = j
                target_position = j + 1
                # print(initial_position, target_position)
                sign, res_state = hopping_operator(state, initial_position, target_position)
                res_state_idx = state_idx_dict[tuple(res_state)]

                if j % 2 == 0:
                    hamiltonian[res_state_idx, i] += v * sign
                else:
                    hamiltonian[res_state_idx, i] += w * sign

            if j - 1 >= 0 and state[j - 1] == 0:
                initial_position = j
                target_position = j - 1
                sign, res_state = hopping_operator(state, initial_position, target_position)
                res_state_idx = state_idx_dict[tuple(res_state)]

                if j % 2 == 0:
                    hamiltonian[res_state_idx, i] += w * sign
                else:
                    hamiltonian[res_state_idx, i] += v * sign
    
    return hamiltonian



In [ ]:
N_sites = 3
tot_sites = N_sites * 2
N_particles = N_sites
v = 1
w = 2

basis_set = basis_set_generator(tot_sites, N_particles)
print(basis_set)
# state_idx_dict = state_idx_mapping(basis_set)
# print(state_idx_dict)
hamiltonian = hamiltonian_matrix_generator(basis_set, tot_sites, v, w)
print(hamiltonian)
print(sp.linalg.ishermitian(hamiltonian))

In [ ]:
for state in basis_set:
    for i in range(tot_sites):
        # initial_site = i
        # target_site_right = i + 1
        # target_site_left = i - 1
        
        if i + 1 <= tot_sites - 1:
            print(f"{state} ---- {i}, {i + 1}: {hopping_operator(state, i, i + 1)}")
        if i - 1 >= 0:
            print(f"{state} ---- {i}, {i - 1}: {hopping_operator(state, i, i - 1)}")


# state = basis_set[1]
# for i in range(tot_sites):
#     print(f"{state} ---- {i}, {i + 1}: {hopping_operator(state, i, i + 1)}")
#     print(f"{state} ---- {i}, {i - 1}: {hopping_operator(state, i, i - 1)}")

In [6]:
print(creation_operator(basis_set[0], 1))
for state in basis_set:
    for j in range(tot_sites):
        # print(f"{state} --- {j} : {creation_operator(state, j)}") 
        print(f"{state} --- {j} : {annihilation_operator(state, j)}") 

(0, None)
[1 1 0 0] --- 0 : (1, array([0, 1, 0, 0]))
[1 1 0 0] --- 1 : (-1, array([1, 0, 0, 0]))
[1 1 0 0] --- 2 : (0, None)
[1 1 0 0] --- 3 : (0, None)
[1 0 1 0] --- 0 : (1, array([0, 0, 1, 0]))
[1 0 1 0] --- 1 : (0, None)
[1 0 1 0] --- 2 : (-1, array([1, 0, 0, 0]))
[1 0 1 0] --- 3 : (0, None)
[1 0 0 1] --- 0 : (1, array([0, 0, 0, 1]))
[1 0 0 1] --- 1 : (0, None)
[1 0 0 1] --- 2 : (0, None)
[1 0 0 1] --- 3 : (-1, array([1, 0, 0, 0]))
[0 1 1 0] --- 0 : (0, None)
[0 1 1 0] --- 1 : (1, array([0, 0, 1, 0]))
[0 1 1 0] --- 2 : (-1, array([0, 1, 0, 0]))
[0 1 1 0] --- 3 : (0, None)
[0 1 0 1] --- 0 : (0, None)
[0 1 0 1] --- 1 : (1, array([0, 0, 0, 1]))
[0 1 0 1] --- 2 : (0, None)
[0 1 0 1] --- 3 : (-1, array([0, 1, 0, 0]))
[0 0 1 1] --- 0 : (0, None)
[0 0 1 1] --- 1 : (0, None)
[0 0 1 1] --- 2 : (1, array([0, 0, 0, 1]))
[0 0 1 1] --- 3 : (-1, array([0, 0, 1, 0]))


In [21]:
print(np.where(basis_set == basis_set[1]))
basis = tuple(basis_set[1].copy())
print(tuple(basis_set[1]) == basis)

(array([0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 5, 5]), array([0, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2]))
True
